In [8]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import pandas as pd
import rasterio.plot

import overpy
import geopandas as gpd
# import ee
from shapely.geometry import Polygon, box
import os
# import geemap
import json
import multiprocessing

from tqdm import tqdm
import time


In [9]:
# os.chdir('/Users/yer1k/Documents/Master/Duke/2023 Fall/Capstone/solar_wastewater/30_result/')
os.chdir('/Users/yer1k/Library/CloudStorage/GoogleDrive-iamtsubasaozora@gmail.com/My Drive/Capstone Project/Data/')

In [10]:
api = overpy.Overpass()

In [11]:

# Fetch all wastewater treatment plants within California's boundary (# changed to Texas)
query = f"""
    area[admin_level=4]["name"="Kansas"]->.searchArea; 
    (
    way["man_made"="wastewater_plant"](area.searchArea);
    );
    (._;>;);
    out body;
    """

result = api.query(query)

plants = {}

for way in result.ways:
    # Use the name of the plant or its ID if the name is not available
    plant_name = way.tags.get("name", f"Plant_{way.id}")

    # Extract nodes lat and lon without id
    nodes_coords = [(node.lon, node.lat) for node in way.nodes]

    # get rid of "Decimal"  in the coordinates
    nodes_coords = [tuple(map(float, i)) for i in nodes_coords]
    
    plants[plant_name] = nodes_coords

# plants_with_official_name = {key: value for key, value in plants.items() if not key.startswith("Plant_")}


geoms = [Polygon(plants[key]) for key in plants]
df = gpd.GeoDataFrame({'WWTP_name': list(plants.keys()), 'geometry': geoms}, crs="EPSG:4326")
df["centroid"] = df.to_crs('+proj=cea').centroid.to_crs(epsg=4326)



In [12]:
df

,WWTP_name,geometry,centroid
0,Plant_93755775,"POLYGON ((-96.73198 39.69701, -96.72904 39.697...",POINT (-96.73048 39.69537)
1,Plant_95756290,"POLYGON ((-96.76414 39.42686, -96.76414 39.426...",POINT (-96.76335 39.42613)
2,Old City Sewer Ponds,"POLYGON ((-100.45767 37.82334, -100.45715 37.8...",POINT (-100.45668 37.82331)
3,Nelson Wastewater Treatment Facility,"POLYGON ((-94.65340 39.04405, -94.65327 39.044...",POINT (-94.65178 39.04213)
4,Harold Street Waste Water Treatment Plant,"POLYGON ((-94.81868 38.89833, -94.81943 38.898...",POINT (-94.81935 38.89899)
...,...,...,...
204,Plant_1197117183,"POLYGON ((-96.41168 39.38042, -96.41093 39.380...",POINT (-96.41097 39.38217)
205,Plant_1202204350,"POLYGON ((-97.76471 39.26176, -97.76098 39.261...",POINT (-97.76256 39.26060)
206,Plant_1202204360,"POLYGON ((-97.84014 39.35032, -97.83804 39.350...",POINT (-97.83908 39.35163)
207,Plant_1202209774,"POLYGON ((-97.52528 39.45022, -97.52527 39.450...",POINT (-97.52415 39.44982)


In [13]:
!ls

Archive                         Status Track.gsheet
PPT From Christine              WWTP Lists
Satellite Imagery_Automated RGB tif_bounding_box


In [14]:
# Added progress bar
# use loop to open the image in tif folder, and plot the bounding box on it
# read all the file names in the folder
directory = 'tif_bounding_box/Images of All 50 States/Kansas'
files = os.listdir(directory)

for file in tqdm(files, desc='plotting bounding box'):
    try:
        with rasterio.open(os.path.join(directory, file)) as src:
            # if file.startswith('download'):
                # continue
            # Read the image data
            img = src.read([1, 2, 3])  # Reads the first three bands for RGB
            # file name without extension
            name = os.path.splitext(file)[0]
            # find the corresponding bounding box
            df_name = df.loc[df['WWTP_name'] == name, :]
            # plot the bounding box
            fig, ax = plt.subplots(figsize=(8,8))
            # Plot the raster
            ax = rasterio.plot.show(img, ax=ax, transform=src.transform)
            # Plot the geometry on the ax axes instance
            df_name.boundary.plot(ax=ax, color='red', linewidth=1.5)
            plt.axis('off')
            # save the figure
            plt.savefig(os.path.join(directory, name + '.png'), bbox_inches='tight', pad_inches=0)
            plt.close()
    except:
        print(f"fail to plot {file}")